In [24]:
import pandas as pd
import numpy as np
from Bio.PDB import MMCIFParser, PDBIO

import pyrosetta
from pyrosetta import init, pose_from_pdb


if not hasattr(pyrosetta, "_already_init"):
    pyrosetta.init("-mute all")
    pyrosetta._already_init = True


def CDRH3_Bfactors(pose):    

    pose_i1 = pyrosetta.rosetta.protocols.antibody.AntibodyInfo(pose)
    cdr_h3_i_first = pyrosetta.rosetta.protocols.antibody.AntibodyInfo.get_CDR_start(pose_i1,pyrosetta.rosetta.protocols.antibody.CDRNameEnum.h3,pose)

    cdr_h3_i_last = pyrosetta.rosetta.protocols.antibody.AntibodyInfo.get_CDR_end(pose_i1,pyrosetta.rosetta.protocols.antibody.CDRNameEnum.h3,pose)
    h3_plddt_array = [pose.pdb_info().temperature(i,1) for i in range (cdr_h3_i_first+1, cdr_h3_i_last+1)]  
    return h3_plddt_array

def CDRL3_Bfactors(pose):    

    pose_i1 = pyrosetta.rosetta.protocols.antibody.AntibodyInfo(pose)
    cdr_l3_i_first = pyrosetta.rosetta.protocols.antibody.AntibodyInfo.get_CDR_start(pose_i1,pyrosetta.rosetta.protocols.antibody.CDRNameEnum.l3,pose)

    cdr_l3_i_last = pyrosetta.rosetta.protocols.antibody.AntibodyInfo.get_CDR_end(pose_i1,pyrosetta.rosetta.protocols.antibody.CDRNameEnum.l3,pose)
    l3_plddt_array = [pose.pdb_info().temperature(i,1) for i in range (cdr_l3_i_first+1, cdr_l3_i_last+1)]  
    return l3_plddt_array



df=pd.read_csv('final_data_with_new_rosetta_metrics_20250521.csv')

# 1) PyRosetta 초기화
init("-mute all")

# 2) mmCIF 읽어서 PDB로 저장
def convert_cif_to_pdb(cif_path: str, pdb_out: str):
    parser = MMCIFParser(QUIET=True)
    structure = parser.get_structure("X", cif_path)
    io = PDBIO()
    io.set_structure(structure)
    io.save(pdb_out)

cif_file = df.loc[1,'pdb']
pdb_file = "/tmp/converted_structure.pdb"
convert_cif_to_pdb(cif_file, pdb_file)

# 3) PyRosetta에 로드
af3_pose = pose_from_pdb(pdb_file)
h3_plddt_array = CDRH3_Bfactors(af3_pose)
l3_plddt_array = CDRL3_Bfactors(af3_pose)

print(np.array(h3_plddt_array).mean())
print(np.array(l3_plddt_array).mean())


pdb_file = df.loc[1,'model_path']

af3_pose = pose_from_pdb(pdb_file)
h3_plddt_array = CDRH3_Bfactors(af3_pose)
l3_plddt_array = CDRL3_Bfactors(af3_pose)

print(np.array(h3_plddt_array).mean())
print(np.array(l3_plddt_array).mean())

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.conda.ubuntu.cxx11thread.serialization.Ubuntu.python310.Release 2024.42+release.3366cf78a3df04339d1982e94531b77b098ddb99 2024-10-11T08:24:04] retrieved from: http://www.pyrosetta.org
97.78285714285714
97.3725


97.78285714285714
97.3725


In [7]:
pdb

'/home/cseomoon/project/ABAG/AbNb_benchmark/AF3_results/8d9z_8d9z/seed-1_sample-1/model.cif'